## Accelerate Inference: Neural Network Pruning

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
from google.colab import files

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.14.0


In [2]:
# mount the drive
from google.colab import drive
drive_path_primitive = "/content/drive/"
drive.mount(drive_path_primitive)

drive_path = drive_path_primitive + "MyDrive/ml-large-datasets-project/"

# use "shutil" to bring data and previous model into local memory
import shutil
src_data = drive_path + "dataset.tar.gz"
shutil.copy(src = src_data, dst = "./dataset.tar.gz")

load_previous_model = True
model_name = "model-start-v1.keras"
# model_name = "model-network-slim-step1of2.keras"
src_model = drive_path + "/" + model_name
shutil.copy(src = src_model, dst = "./" + model_name)

Mounted at /content/drive/


'./model-start-v1.keras'

In [3]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [4]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

print(model.summary())
total_trainable_params_original = 592933

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [ ]:
if load_previous_model:
  # load previous model from the working directory
  # note that the model should be uploaded to the working directory
  model = keras.models.load_model(model_name)
else:
  # you can use the default hyper-parameters for training,
  # val accuracy ~72% after 50 epochs

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])

  history = model.fit(train_images, train_labels, batch_size=32, epochs=50,
                      validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

In [ ]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 3s 17ms/step - loss: 0.7373 - accuracy: 0.7180


In [ ]:
# perform pruning here - SEE ALL CELLS BELOW

# get the weights
weights = model.get_weights()

# you can use set_weights() to set some weights to zero, e.g.,

weights[7][:10]=0
model.set_weights(weights)


In [ ]:
# temporarily add intervening layers for batch normalization, see the
# "Leveraging the Scaling Factors in BN Layers" section in paper

# Define the neural network architecture (don't change this)
temp_model = models.Sequential()
temp_model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
i = 0
temp_model.add(BatchNormalization(gamma_regularizer=regularizers.l1(1e-5), center=False,name=f"temp_layer_{i}"))
i += 1
temp_model.add(Activation('relu'))
temp_model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
temp_model.add(BatchNormalization(gamma_regularizer=regularizers.l1(1e-5), center=False,name=f"temp_layer_{i}"))
i += 1
temp_model.add(Activation('relu'))
temp_model.add(MaxPooling2D(pool_size=(2, 2)))
temp_model.add(Dropout(0.25))
temp_model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
temp_model.add(BatchNormalization(gamma_regularizer=regularizers.l1(1e-5), center=False,name=f"temp_layer_{i}"))
i += 1
temp_model.add(Activation('relu'))
temp_model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
temp_model.add(BatchNormalization(gamma_regularizer=regularizers.l1(1e-5), center=False,name=f"temp_layer_{i}"))
i += 1
temp_model.add(Activation('relu'))
temp_model.add(MaxPooling2D(pool_size=(2, 2)))
temp_model.add(Dropout(0.25))
temp_model.add(Flatten())
temp_model.add(Dense(512))
temp_model.add(BatchNormalization(gamma_regularizer=regularizers.l1(1e-5), center=False,name=f"temp_layer_{i}"))
i += 1
temp_model.add(Activation('relu'))
temp_model.add(Dropout(0.5))
temp_model.add(Dense(5))
temp_model.add(BatchNormalization(gamma_regularizer=regularizers.l1(1e-5), center=False,name=f"temp_layer_{i}"))
i += 1
temp_model.add(Activation('softmax'))

In [ ]:
number_batch_norm = 6
check = 0
for layer_idx, layer in enumerate(temp_model.layers):
  # print(layer_idx, layer)
  if "temp" in layer.name:
    check += 1
assert check == number_batch_norm, f"There should be {number_batch_norm} layers!"

In [ ]:
print(temp_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 25, 25, 32)        896       
                                                                 
 temp_layer_0 (BatchNormali  (None, 25, 25, 32)        96        
 zation)                                                         
                                                                 
 activation_6 (Activation)   (None, 25, 25, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 temp_layer_1 (BatchNormali  (None, 23, 23, 32)        96        
 zation)                                                         
                                                                 
 activation_7 (Activation)   (None, 23, 23, 32)       

In [ ]:
load_previous_temp_model = True
previous_temp_model_name = "temp-model.keras"

if not load_previous_temp_model:

  # re-train the temporary model, leveraging the Batch Normalization layers
  # to get the channel scaling factors
  # "Leveraging the Scaling Factors in BN Layers" section in paper

  temp_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])

  history_temp = temp_model.fit(train_images, train_labels, batch_size=32, epochs=50,
                        validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32
else:
  # copy the temp_model from drive into the local memory and load the model
  src_temp_model = drive_path + "/" + previous_temp_model_name
  shutil.copy(src = src_temp_model, dst = "./" + previous_temp_model_name)
  temp_model = keras.models.load_model(previous_temp_model_name)

In [ ]:
# GOAL: identify channels which need pruned for each layer in the original model
ready_to_prune_1 = False
ready_to_prune_2 = False
for layer_idx, layer in enumerate(temp_model.layers):
  if "temp" not in layer.name:
    # print(layer)
    layer_to_prune = layer
    ready_to_prune_1 = True
  elif  "temp" in layer.name:
    # print(layer)
    batch_norm_layer = layer
    ready_to_prune_2 = True

  # see if we've collected both pairs
  if ready_to_prune_1 and ready_to_prune_2:
      print(layer_to_prune.name,batch_norm_layer.name)
      # reset it
      ready_to_prune_1 = False
      ready_to_prune_2 = False


conv2d_4 temp_layer_0
conv2d_5 temp_layer_1
conv2d_6 temp_layer_2
conv2d_7 temp_layer_3
dense_2 temp_layer_4
dense_3 temp_layer_5


In [ ]:
# GOAL: identify channels which need pruned for each layer in the original model
# STEP 1: calculate global percentile bins.
# 1a. get min and max

# initialize them
max_gamma = -np.inf
min_gamma = np.inf

for layer_idx, layer in enumerate(temp_model.layers):
  # print(temp_model.layers[layer_idx-1].name)

  if  "temp" in layer.name and "dense_2" == temp_model.layers[layer_idx-1].name:
  # if  "temp" in layer.name and "dense_3" != temp_model.layers[layer_idx-1].name:
    # print(layer.name)
    batch_norm_layer = layer

    # https://stackoverflow.com/questions/42521005/how-the-number-of-parameters-associated-with-batchnormalization-layer-is-2048
    gamma_weights = batch_norm_layer.weights[0]

    # https://stackoverflow.com/questions/70043645/how-to-convert-from-tensor-to-float
    layer_max_gamma = tf.reduce_max(gamma_weights).numpy()
    layer_min_gamma = tf.reduce_min(gamma_weights).numpy()

    if layer_max_gamma > max_gamma:
      max_gamma = layer_max_gamma

    if layer_min_gamma < min_gamma:
      min_gamma = layer_min_gamma
  # elif "temp" in layer.name and "dense_2" not in layer.name:


print(min_gamma,max_gamma)

0.9195836 1.0561795


In [ ]:
# step 1b. calculate the percentiles

q = np.arange(0,100,1)

print(q)

percentiles = np.percentile(a = [min_gamma,max_gamma], q=q)

print(percentiles)
print(len(percentiles))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[0.91958362 0.92094958 0.92231554 0.9236815  0.92504745 0.92641341
 0.92777937 0.92914533 0.93051129 0.93187725 0.93324321 0.93460917
 0.93597513 0.93734109 0.93870705 0.940073   0.94143896 0.94280492
 0.94417088 0.94553684 0.9469028  0.94826876 0.94963472 0.95100068
 0.95236664 0.95373259 0.95509855 0.95646451 0.95783047 0.95919643
 0.96056239 0.96192835 0.96329431 0.96466027 0.96602623 0.96739219
 0.96875814 0.9701241  0.97149006 0.97285602 0.97422198 0.97558794
 0.9769539  0.97831986 0.97968582 0.98105178 0.98241773 0.98378369
 0.98514965 0.98651561 0.98788157 0.98924753 0.99061349 0.99197945
 0.99334541 0.99471137 0.99607733 0.99744328 0.99880924 1.0001752
 1.00154116 1.00290712 1.

In [ ]:
# METHOD FOR GENERATING A SINGLE PERCENTILE

desired_percentile = 59

idx_desired_percentile = np.where(q==desired_percentile)

cutoff_gamma = percentiles[idx_desired_percentile]
print("cutoff_gamma = ",cutoff_gamma)

cutoff_gamma =  [1.0001752]


In [ ]:
# step 1b. calculate the percentiles
num_cutoffs_desired = 10

all_cutoffs = np.linspace(min_gamma,max_gamma,num_cutoffs_desired)

print("all_cutoffs",all_cutoffs)

all_cutoffs [0.91958362 0.93476094 0.94993826 0.96511559 0.98029291 0.99547023
 1.01064756 1.02582488 1.0410022  1.05617952]


In [ ]:
sparsity_vs_accuracy_list = []

# STEP 2: using the thresholds from above, perform pruning on the channels
retrain_pruned_model_bool = True
number_retraining_epochs = 10
for cutoff_gamma in all_cutoffs:
  print("cutoff_gamma = ", cutoff_gamma)

  if load_previous_temp_model:
    temp_model = keras.models.load_model(previous_temp_model_name)
  else:
    print("""BE SURE TO RE-GENERATE ORIGINAL TEMP MODEL BEFORE THIS,
          if you want to see the actual before and after calcs!!!""")

  # GOAL: identify channels which need pruned for each layer in the original model
  # Step 2: prune based on the selected percentile
  ready_to_prune_1 = False
  ready_to_prune_2 = False
  total_pruned = 0
  for layer_idx, layer in enumerate(temp_model.layers):
    if "temp" not in layer.name:
      # print(layer)
      layer_to_prune = layer
      ready_to_prune_1 = True
    elif  "temp" in layer.name:
      # print(layer)
      batch_norm_layer = layer
      ready_to_prune_2 = True

    # see if we've collected both pairs, if so, calculate the pruned weights
    if ready_to_prune_1 and ready_to_prune_2:
        weights_to_prune, bias_to_prune = layer_to_prune.get_weights()
        bn_gamma_weights = batch_norm_layer.weights[0]

        # BELOW REVEALS THAT the last Dense layer contains the max value
        # print("tf.reduce_max(bn_gamma_weights)",float(tf.reduce_max(bn_gamma_weights)))

        channels_to_prune_idxs = np.where(bn_gamma_weights<cutoff_gamma)[0]
        # print(channels_to_prune_idxs)

        nonzero_params_before = np.count_nonzero(weights_to_prune) + np.count_nonzero(bias_to_prune)
        print("nonzero_params_before",nonzero_params_before)

        for channel_idx in channels_to_prune_idxs:
          # handle conv2D
          if len(weights_to_prune.shape) == 4:
            # print("WE SAW THIS")
            weights_to_prune[:,:,:,channel_idx] = 0
            bias_to_prune[channel_idx] = 0
          # handle Dense Layer
          else:
            # print("WE SAW THIS")
            weights_to_prune[:,channel_idx] = 0
            bias_to_prune[channel_idx] = 0

        nonzero_params_after = np.count_nonzero(weights_to_prune) + np.count_nonzero(bias_to_prune)
        print("nonzero_params_after",nonzero_params_after)

        total_pruned += nonzero_params_before - nonzero_params_after


        layer_to_prune.set_weights([weights_to_prune,bias_to_prune])
        # print(channels_to_prune_idxs.shape)
        # raise Exception("stop")
        # print("original weights shape",[weight.shape for weight in layer_to_prune.weights])
        # print("temp weights shape",[weight.shape for weight in batch_norm_layer.weights])
        # print("temp weights shape",[weight[0:5] for weight in batch_norm_layer.weights])
        # print()
        # print(layer_to_prune.name,batch_norm_layer.name)
        # raise Exception("STOP")
        # print(batch_norm_layer.weights)

        # reset it
        ready_to_prune_1 = False
        ready_to_prune_2 = False


  print("total_pruned:",total_pruned)
  print("total_trainable_params_original:",total_trainable_params_original)
  print("fraction pruned:",total_pruned/total_trainable_params_original)

  # perform the pruning using the current cutoff
  new_model_weights = []
  new_weights = temp_model.get_weights()
  for l_idx, l in enumerate(new_weights):
    if l_idx % 5 < 2:
        new_model_weights.append(l)
  # for m in new_model_weights:
  #   print(m.shape)

  model.set_weights(new_model_weights)

  # evaluate again to see how the accuracy changes
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

  if retrain_pruned_model_bool:
    # retrain for specified number of epochs, with batch size 32
    history = model.fit(train_images, train_labels, batch_size=32, epochs=number_retraining_epochs,
                          validation_data=(val_images, val_labels))

    total_nonzero_after_retraining = sum([np.count_nonzero(weight_vec) for weight_vec in model.weights])
    total_pruned = total_trainable_params_original - total_nonzero_after_retraining
    sparsity = total_pruned/total_trainable_params_original
    print("total_pruned, after retraining pruned:",total_pruned)
    print("total_trainable_params_original, after retraining pruned:",total_trainable_params_original)
    print("sparsity, after retraining pruned:",sparsity)
  else:
    total_nonzero_after_retraining = sum([np.count_nonzero(weight_vec) for weight_vec in model.weights])
    total_pruned = total_trainable_params_original - total_nonzero_after_retraining
    sparsity = total_pruned/total_trainable_params_original
    print("total_pruned, after NOT retraining pruned:",total_pruned)
    print("total_trainable_params_original, after NOT retraining pruned:",total_trainable_params_original)
    print("sparsity, after NOT retraining pruned:",sparsity)

  # you need to save the model's weights, naming it 'my_model_weights.h5'
  save_name = f"my_model_weights_network_slimming_cutoff_gamma_{cutoff_gamma}.h5"
  model.save_weights(save_name)
  files.download(save_name)

  # evaluate again to see how the accuracy changes
  results = model.evaluate(val_images, val_labels, batch_size=128)
  accuracy = results[1]

  save_tuple = (accuracy, sparsity, cutoff_gamma)
  sparsity_vs_accuracy_list.append(save_tuple)


  print("finished cutoff_gamma = ", cutoff_gamma)
  print("________________________________________________")


cutoff_gamma =  0.9195836186408997
nonzero_params_before 896
nonzero_params_after 868
nonzero_params_before 9248
nonzero_params_after 8959
nonzero_params_before 18496
nonzero_params_after 17918
nonzero_params_before 36928
nonzero_params_after 36351
nonzero_params_before 524800
nonzero_params_after 524800
nonzero_params_before 2565
nonzero_params_after 2565
total_pruned: 1472
total_trainable_params_original: 592933
fraction pruned: 0.0024825739164458715
Epoch 1/10
703/703 [==============================] - 7s 8ms/step - loss: 1.1003 - accuracy: 0.5669 - val_loss: 0.9273 - val_accuracy: 0.6507
Epoch 2/10
703/703 [==============================] - 4s 6ms/step - loss: 0.8507 - accuracy: 0.6800 - val_loss: 0.8241 - val_accuracy: 0.6867
Epoch 3/10
703/703 [==============================] - 4s 6ms/step - loss: 0.7952 - accuracy: 0.7036 - val_loss: 0.8107 - val_accuracy: 0.6879
Epoch 4/10
703/703 [==============================] - 5s 7ms/step - loss: 0.7595 - accuracy: 0.7151 - val_loss: 0.732

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

20/20 [==============================] - 0s 4ms/step - loss: 0.7259 - accuracy: 0.7251
finished cutoff_gamma =  0.9195836186408997
________________________________________________
cutoff_gamma =  0.9347609413994683
nonzero_params_before 896
nonzero_params_after 812
nonzero_params_before 9248
nonzero_params_after 8670
nonzero_params_before 18496
nonzero_params_after 17918
nonzero_params_before 36928
nonzero_params_after 35774
nonzero_params_before 524800
nonzero_params_after 522750
nonzero_params_before 2565
nonzero_params_after 2565
total_pruned: 4444
total_trainable_params_original: 592933
fraction pruned: 0.00749494462274827
Epoch 1/10
703/703 [==============================] - 8s 8ms/step - loss: 1.1135 - accuracy: 0.5602 - val_loss: 0.9200 - val_accuracy: 0.6451
Epoch 2/10
703/703 [==============================] - 4s 6ms/step - loss: 0.8622 - accuracy: 0.6769 - val_loss: 0.8309 - val_accuracy: 0.6931
Epoch 3/10
703/703 [==============================] - 4s 6ms/step - loss: 0.8136 

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

20/20 [==============================] - 0s 4ms/step - loss: 1.4986 - accuracy: 0.3418
finished cutoff_gamma =  1.0410022007094488
________________________________________________
cutoff_gamma =  1.0561795234680176
nonzero_params_before 896
nonzero_params_after 140
nonzero_params_before 9248
nonzero_params_after 578
nonzero_params_before 18496
nonzero_params_after 1156
nonzero_params_before 36928
nonzero_params_after 1154
nonzero_params_before 524800
nonzero_params_after 1025
nonzero_params_before 2565
nonzero_params_after 2565
total_pruned: 586315
total_trainable_params_original: 592933
fraction pruned: 0.988838536563153
Epoch 1/10
703/703 [==============================] - 8s 8ms/step - loss: 1.6095 - accuracy: 0.1963 - val_loss: 1.6095 - val_accuracy: 0.2115
Epoch 2/10
703/703 [==============================] - 4s 6ms/step - loss: 1.6093 - accuracy: 0.2153 - val_loss: 1.6090 - val_accuracy: 0.2321
Epoch 3/10
703/703 [==============================] - 4s 6ms/step - loss: 1.6074 - acc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

20/20 [==============================] - 0s 5ms/step - loss: 1.5899 - accuracy: 0.2677
finished cutoff_gamma =  1.0561795234680176
________________________________________________


In [ ]:
sparsity_vs_accuracy_list

[(0.7251484990119934, 0.0024825739164458715, 0.9195836186408997),
 (0.7302970290184021, 0.00749494462274827, 0.9347609413994683),
 (0.7223762273788452, 0.0662671836446951, 0.949938264158037),
 (0.6986138820648193, 0.19055609993034625, 0.9651155869166056),
 (0.6867326498031616, 0.4045448642595369, 0.9802929096751742),
 (0.6257425546646118, 0.6091868727158043, 0.995470232433743),
 (0.5013861656188965, 0.8197098154428916, 1.0106475551923115),
 (0.36752474308013916, 0.9310427991020908, 1.0258248779508803),
 (0.34178218245506287, 0.9669777192364062, 1.0410022007094488),
 (0.2677227854728699, 0.988838536563153, 1.0561795234680176)]

In [ ]:
np.array(sparsity_vs_accuracy_list)

array([[0.7251485 , 0.00248257, 0.91958362],
       [0.73029703, 0.00749494, 0.93476094],
       [0.72237623, 0.06626718, 0.94993826],
       [0.69861388, 0.1905561 , 0.96511559],
       [0.68673265, 0.40454486, 0.98029291],
       [0.62574255, 0.60918687, 0.99547023],
       [0.50138617, 0.81970982, 1.01064756],
       [0.36752474, 0.9310428 , 1.02582488],
       [0.34178218, 0.96697772, 1.0410022 ],
       [0.26772279, 0.98883854, 1.05617952]])

In [ ]:
np.savetxt("sparsity_vs_accuracy_list.txt",np.array(sparsity_vs_accuracy_list))
files.download("sparsity_vs_accuracy_list.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(type(weights))
print([weights[i].shape for i in range(len(weights))])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 25, 25, 32)        896       
                                                                 
 activation_6 (Activation)   (None, 25, 25, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 11, 11, 32)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 11, 11, 32)        0         
                                                      

In [ ]:
# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 7s 13ms/step - loss: 0.7373 - accuracy: 0.7180


In [ ]:
results[1]

0.7180197834968567

In [ ]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("my_model_weights.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights.h5")

cutoff_gamma = 10
save_name = f"my_model_weights_network_slimming_cutoff_gamma_{cutoff_gamma}.h5"
model.save_weights(save_name)
files.download(save_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#                 metrics=['accuracy'])


# model.load_weights('./my_model_weights_2.h5', skip_mismatch=False)

# # evaluate again to see how the accuracy changes
# results = model.evaluate(val_images, val_labels, batch_size=128)
# accuracy = results[1]

20/20 [==============================] - 7s 11ms/step - loss: 0.7970 - accuracy: 0.6966


In [8]:

# num_weights = 0
# num_nonzeros = 0
# for t in model.get_weights():
#    num_weights += np.prod(t.shape)
#    num_nonzeros += np.count_nonzero(t)


# sparse = 1 - num_nonzeros/num_weights
# print(sparse)

0.6817448177112759
